In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import praw
from datetime import datetime, timedelta
from prawcore.exceptions import Redirect

user_agent = "Scraper 1.0 by AmphibianFinal9736"
reddit = praw.Reddit(client_id="HUmX-XMSMg7y2lnN-O21DA", client_secret="e2w9HmBAsBOPwZvuHDDykxsm12PKIA", user_agent=user_agent, password="Wangyibo@11")

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from prawcore.exceptions import Redirect

from collections import Counter
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/varalam/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/varalam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varalam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def fetch_comments_and_submissions(subreddit_names, keywords, limit_per_subreddit=100):
    sia = SIA()

    # List to store comments and submissions
    results = []

    for subreddit_name in subreddit_names:
        subreddit = reddit.subreddit(subreddit_name)

        print(f"\nFetching comments and submissions from r/{subreddit_name}...\n{'=' * 40}")

        try:
            # Search for submissions containing the keywords
            for submission in subreddit.search(" OR ".join(keywords), sort="new", time_filter="all", limit=limit_per_subreddit):
                # Fetch comments from the submission
                submission.comments.replace_more(limit=50)  # Retrieve all comments (not just top-level)
                for comment in submission.comments.list():
                    # Print each comment's body (you can modify this part based on your requirements)
                    # print(f"Comment: {comment.body}")
                    
                    # Perform sentiment analysis on comments
                    pol_score_comment = sia.polarity_scores(comment.body)
                    pol_score_comment['comments'] = comment.body
                    results.append(pol_score_comment)

                # Print submission title and URL
                print(f"Submission Title: {submission.title}")
                print(f"Submission URL: {submission.url}")
                print("-" * 40)

        except Redirect as e:
            print(f"Redirect exception: {e}")
            continue

    return results



In [18]:
def process_comments(df):
    # Create a new DataFrame to store the processed results
    processed_df = df.copy()

    # Add a 'label' column based on the 'compound' score
    processed_df['label'] = 0
    processed_df.loc[processed_df['compound'] > 0.2, 'label'] = 1
    processed_df.loc[processed_df['compound'] < -0.2, 'label'] = -1

    # Tokenize the comments into words
    processed_df['tokens'] = processed_df['comments'].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    processed_df['filtered_tokens'] = processed_df['tokens'].apply(
        lambda tokens: [word for word in tokens if word.lower() not in stop_words]
    )

    # Remove punctuation
    processed_df['filtered_tokens'] = processed_df['filtered_tokens'].apply(
        lambda tokens: [word for word in tokens if word.isalnum()]
    )

    # Additional words to remove
    words_to_remove = ['Lamborghini', 'Lambo', 'car', 'lambo', 'lamborghini', 'huracan', 'aventador', 'countach', 'new',
                       'diablo', 'urus', 'cars', 'https']
    processed_df['filtered_tokens'] = processed_df['filtered_tokens'].apply(
        lambda tokens: [word.lower() for word in tokens if word.lower() not in words_to_remove]
    )

    
    # Save the DataFrame to a CSV file with tokenized words
    processed_df[['comments', 'filtered_tokens']].to_csv('TwitchPromotion_comments.csv', index=False)

    # Flatten the list of filtered tokens
    all_words = [word for tokens in processed_df['filtered_tokens'] for word in tokens]
    
    # Count the occurrences of each word
    word_counts = Counter(all_words)

    # Get the most common words and their frequencies
    most_common_words = word_counts.most_common()

    # Create a DataFrame for most common words
    most_common_words_df = pd.DataFrame(most_common_words, columns=['Word', 'Frequency'])

    
    return processed_df, most_common_words_df



In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def generate_topics(df, num_topics=20):
    # List of Lamborghini car models to be removed
    lamborghini_models = [
        "Huracan", "Aventador", "Urus", "Gallardo", "Murcielago", "Diablo",
        "Countach", "Jalpa", "Espada", "Islero", "Jarama", "Silhouette",
        "Urraco", "Miura", "Reventon", "Centenario", "Sian", 'personam', 'Revuelto', 'svj', 'lambo'
        # Add more Lamborghini models as needed
    ]

    # Convert Lamborghini car models to lowercase
    lamborghini_models_lower = [model.lower() for model in lamborghini_models]

    # Tokenization and preprocessing
    stop_words = set(stopwords.words('english'))
    comments_doc = df['comments']
    comments_doc = [word_tokenize(doc.lower()) for doc in comments_doc]
    comments_doc = [
        [word for word in doc if word.isalnum() and word not in stop_words and word not in lamborghini_models_lower]
        for doc in comments_doc
    ]

    # Create TF-IDF matrix
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in comments_doc])

    # Apply Truncated SVD (LSA)
    lsa_model = TruncatedSVD(n_components=num_topics)
    lsa_topic_matrix = lsa_model.fit_transform(tfidf_matrix)

    # Print the top words for each topic
    terms = vectorizer.get_feature_names_out()
    topics = {}
    for i in range(num_topics):
        top_words_idx = lsa_model.components_[i].argsort()[-5:][::-1]
        top_words = [terms[idx] for idx in top_words_idx]
        topics[f'Topic {i + 1}'] = top_words

    return topics




In [5]:
import gensim
from gensim import corpora
from gensim.models import LdaModel


def train_lda_model(documents, num_topics=20, num_words=5, passes=10):
    # Tokenization and preprocessing
    stop_words = set(stopwords.words('english'))
    documents = [word_tokenize(doc.lower()) for doc in documents]
    documents = [[word for word in doc if word.isalnum() and word not in stop_words] for doc in documents]

    # Create a dictionary representation of the documents
    dictionary = corpora.Dictionary(documents)

    # Create a corpus from the documents
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    # Train the LDA model
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

    # Print the topics
    topics = lda_model.print_topics(num_words=num_words)
    formatted_topics = []

    for topic_id, topic in topics:
        terms = [term.split('"')[1] for term in topic.split('+')]
        formatted_topic = f"Topic {topic_id}: {' + '.join(terms)}"
        formatted_topics.append(formatted_topic)

    return formatted_topics



In [ ]:
# Set the subreddit and keyword
subreddit_names = ['lamborghini']
keywords = ['games', 'game', 'GAME', 'videogames', 'racinggame', 'video games', 'raced', 'race', 'league', 'tournament', 'collab', 'collaboration', 'asia', 'singapore']

comments_and_submissions_results = fetch_comments_and_submissions(subreddit_names, keywords, limit_per_subreddit=None)

# Create a DataFrame from comments_and_submissions_results
comments_df = pd.DataFrame.from_records(comments_and_submissions_results)

# Print the DataFrame
print("Comments and Submissions DataFrame:")
print(comments_df)

# # Example usage:
# # Assuming you have a DataFrame named comments_df with a 'comments' column
# processed_comments_df, common_words_comments = process_comments(comments_df)
# print("Processed Comments DataFrame:")
# print(processed_comments_df.head())

# print("\nMost Common Words in Comments:")
# print(common_words_comments.head(50))


# Example usage:
# Assuming you have a DataFrame named commentss_df with a 'comments' column
topics = generate_topics(comments_df, num_topics=20)
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')

# LDA model:
headline_topics = train_lda_model(comments_df['comments'])
for topic in headline_topics:
    print(topic)



In [16]:
# Set the subreddit and keyword
subreddit_names = ['simracing']
keywords = ['games', 'game', 'GAME', 'videogames', 'racinggame', 'video games', 'raced', 'race', 'league', 'tournament', 'collab', 'collaboration', 'asia', 'singapore', 'Singapore']


comments_and_submissions_results = fetch_comments_and_submissions(subreddit_names, keywords, limit_per_subreddit=None)

# Create a DataFrame from comments_and_submissions_results
comments_df = pd.DataFrame.from_records(comments_and_submissions_results)

# Print the DataFrame
print("Comments and Submissions DataFrame:")
print(comments_df)

# Example usage:
# Assuming you have a DataFrame named comments_df with a 'comments' column
processed_comments_df, common_words_comments = process_comments(comments_df)
print("Processed Comments DataFrame:")
print(processed_comments_df.head())

print("\nMost Common Words in Comments:")
print(common_words_comments.head(50))


# Example usage:
# Assuming you have a DataFrame named commentss_df with a 'comments' column
topics = generate_topics(comments_df, num_topics=20)
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')

# LDA model:
headline_topics = train_lda_model(comments_df['comments'])
for topic in headline_topics:
    print(topic)





Fetching comments and submissions from r/simracing...
Submission Title: Used bundle for 1100 euros
Submission URL: https://www.reddit.com/r/simracing/comments/1adr75n/used_bundle_for_1100_euros/
----------------------------------------
Submission Title: Potential for a Mille Miglia inspired track?
Submission URL: https://www.reddit.com/r/simracing/comments/1adppo6/potential_for_a_mille_miglia_inspired_track/
----------------------------------------
Submission Title: Solving MOZA R5 overheating / Cooling MOZA R5
Submission URL: https://www.reddit.com/r/simracing/comments/1adpa85/solving_moza_r5_overheating_cooling_moza_r5/
----------------------------------------
Submission Title: Seats with great lumbar support?
Submission URL: https://www.reddit.com/r/simracing/comments/1adnffz/seats_with_great_lumbar_support/
----------------------------------------
Submission Title: Single 34 inch vs working up to triple 27' monitors
Submission URL: https://www.reddit.com/r/simracing/comments/1adml

Submission Title: Can anyone explain this to me. Trophi ai trojan?
Submission URL: https://i.redd.it/ax0qwil1e2fc1.jpeg
----------------------------------------
Submission Title: SimXPro and MOZA compatibility
Submission URL: https://www.reddit.com/r/simracing/comments/1acno3e/simxpro_and_moza_compatibility/
----------------------------------------
Submission Title: iracing still is were its at for big playerbase and good online play?
Submission URL: https://www.reddit.com/r/simracing/comments/1acn0sl/iracing_still_is_were_its_at_for_big_playerbase/
----------------------------------------
Submission Title: Asetek Front Mount Issue
Submission URL: https://www.reddit.com/gallery/1acmarn
----------------------------------------
Submission Title: Daytona 24!!!
Submission URL: https://www.reddit.com/gallery/1aclzgc
----------------------------------------
Submission Title: Decent setup ideas when lacking space?
Submission URL: https://www.reddit.com/r/simracing/comments/1aclvy3/decent_setu

Submission Title: Fourth Monitor Help
Submission URL: https://www.reddit.com/gallery/1abnuyv
----------------------------------------
Submission Title: Simracing in VR - looking for information please !
Submission URL: https://www.reddit.com/r/simracing/comments/1abnssf/simracing_in_vr_looking_for_information_please/
----------------------------------------
Submission Title: What is the lowest budget for a gaming PC, sufficient for AC & iRacing?
Submission URL: https://www.reddit.com/r/simracing/comments/1abnsiy/what_is_the_lowest_budget_for_a_gaming_pc/
----------------------------------------
Submission Title: G29 Accelerator Pedal "Flickering"
Submission URL: https://www.reddit.com/r/simracing/comments/1abm6ei/g29_accelerator_pedal_flickering/
----------------------------------------
Submission Title: What are the recommended Specs for one monitor 1080p to 4k triple screen, is there a chart or list for comparison?
Submission URL: https://www.reddit.com/r/simracing/comments/1abl558/w

Submission Title: Will a Moza R12 base fit on a ASR1 rig??
Submission URL: https://i.redd.it/c3qeszg43lec1.jpeg
----------------------------------------
Submission Title: Sim racing saved me in real life
Submission URL: https://www.reddit.com/r/simracing/comments/19f7iky/sim_racing_saved_me_in_real_life/
----------------------------------------
Submission Title: How to setup triples of differing sizes.
Submission URL: https://www.reddit.com/r/simracing/comments/19f6wbz/how_to_setup_triples_of_differing_sizes/
----------------------------------------
Submission Title: Thoughts on a rig with a VR headset instead of screen?
Submission URL: https://www.reddit.com/r/simracing/comments/19f6i40/thoughts_on_a_rig_with_a_vr_headset_instead_of/
----------------------------------------
Submission Title: Rig wont work with some games
Submission URL: https://www.reddit.com/r/simracing/comments/19f6bex/rig_wont_work_with_some_games/
----------------------------------------
Submission Title: Best sim

Submission Title: Looking to get a better setup
Submission URL: https://www.reddit.com/r/simracing/comments/19ee963/looking_to_get_a_better_setup/
----------------------------------------
Submission Title: Should've started sooner
Submission URL: https://www.reddit.com/r/simracing/comments/19eduiu/shouldve_started_sooner/
----------------------------------------
Submission Title: G923 or T248
Submission URL: https://www.reddit.com/r/simracing/comments/19edgtd/g923_or_t248/
----------------------------------------
Submission Title: Rumble motors and buttkicker
Submission URL: https://www.reddit.com/r/simracing/comments/19ebdn0/rumble_motors_and_buttkicker/
----------------------------------------
Submission Title: Throttle showing as both X and Y axis in USB Game Controllers (like it was a joystick)
Submission URL: https://www.reddit.com/r/simracing/comments/19eb89c/throttle_showing_as_both_x_and_y_axis_in_usb_game/
----------------------------------------
Submission Title: Fanatech vs 

Submission Title: Having a hard time trying to get into sim racing
Submission URL: https://www.reddit.com/r/simracing/comments/19dg9pb/having_a_hard_time_trying_to_get_into_sim_racing/
----------------------------------------
Submission Title: Rate the setup
Submission URL: https://www.reddit.com/gallery/19dfgg3
----------------------------------------
Submission Title: Mega thread under construction, best intro racing setup?
Submission URL: https://www.reddit.com/r/simracing/comments/19de76o/mega_thread_under_construction_best_intro_racing/
----------------------------------------
Submission Title: Need help and suggestion on locally-made sim rig that I am planning to buy.
Submission URL: https://www.reddit.com/r/simracing/comments/19ddzy9/need_help_and_suggestion_on_locallymade_sim_rig/
----------------------------------------
Submission Title: Question about steering wheels
Submission URL: https://www.reddit.com/r/simracing/comments/19dcccp/question_about_steering_wheels/
----------

Topic 1: ['like', 'wheel', 'get', 'racing', 'sim']
Topic 2: ['thanks', 'bro', 'look', 'mate', 'try']
Topic 3: ['https', 'option', 'great', 'another', 'rig']
Topic 4: ['thank', 'much', 'looking', 'exactly', 'try']
Topic 5: ['pc', 'sim', 'racing', 'rig', 'dedicated']
Topic 6: ['wheel', 'pc', 'rig', 'dedicated', 'sim']
Topic 7: ['wheel', 'racing', 'sim', 'car', 'real']
Topic 8: ['yes', 'vr', 'suck', 'get', 'pc']
Topic 9: ['gif', 'one', 'suck', 'life', 'bro']
Topic 10: ['yes', 'looks', 'like', 'suck', 'rig']
Topic 11: ['deleted', 'know', 'great', 'lowercase', 'racing']
Topic 12: ['good', 'pc', 'looks', 'better', 'much']
Topic 13: ['rig', 'good', 'suck', 'seat', 'dd']
Topic 14: ['know', 'good', 'lol', 'pc', 'one']
Topic 15: ['lol', 'one', 'good', 'got', 'racing']
Topic 16: ['much', 'rig', 'would', 'got', 'good']
Topic 17: ['much', 'got', 'like', 'sure', 'pc']
Topic 18: ['one', 'would', 'price', 'buy', 'fun']
Topic 19: ['one', 'get', 'yeah', 'good', 'go']
Topic 20: ['suck', 'fun', 'money', '

In [7]:
# Set the subreddit and keyword
subreddit_names = ['SgGamers']
keywords = ['games', 'game', 'GAME', 'videogames', 'racinggame', 'video games', 'raced', 'race', 'league', 'tournament']


comments_and_submissions_results = fetch_comments_and_submissions(subreddit_names, keywords, limit_per_subreddit=None)

# Create a DataFrame from comments_and_submissions_results
comments_df = pd.DataFrame.from_records(comments_and_submissions_results)

# Print the DataFrame
print("Comments and Submissions DataFrame:")
print(comments_df)

# Example usage:
# Assuming you have a DataFrame named comments_df with a 'comments' column
processed_comments_df, common_words_comments = process_comments(comments_df)
print("Processed Comments DataFrame:")
print(processed_comments_df.head())

print("\nMost Common Words in Comments:")
print(common_words_comments.head(50))


# Example usage:
# Assuming you have a DataFrame named commentss_df with a 'comments' column
topics = generate_topics(comments_df, num_topics=20)
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')

# LDA model:
headline_topics = train_lda_model(comments_df['comments'])
for topic in headline_topics:
    print(topic)





Fetching comments and submissions from r/SgGamers...
Submission Title: Join us this Christmas at Zolaz Arena and game your way to a sleigh-cation!
Submission URL: https://www.reddit.com/r/SgGamers/comments/18fmjnk/join_us_this_christmas_at_zolaz_arena_and_game/
----------------------------------------
Submission Title: We're launching a game and we would love to get some singaoperan advices
Submission URL: https://www.reddit.com/r/SgGamers/comments/15tjbub/were_launching_a_game_and_we_would_love_to_get/
----------------------------------------
Submission Title: VR Headsets
Submission URL: https://www.reddit.com/r/SgGamers/comments/1369zjc/vr_headsets/
----------------------------------------
Submission Title: discord group for sg gamers
Submission URL: https://www.reddit.com/r/SgGamers/comments/uixtbq/discord_group_for_sg_gamers/
----------------------------------------
Submission Title: Kill a giant while drinking boba tea in Cuisineer, a Singaporean Indie Game Coming to Steam
Submis

Submission Title: Wtf!? A winemaking game made in Singapore!
Submission URL: https://virtualseasia.com/2017/05/25/terroir-review-ea/
----------------------------------------
Submission Title: SPOUT 360 Content Team AMA Cum Fireside chat #1
Submission URL: https://www.reddit.com/r/SgGamers/comments/6abjgm/spout_360_content_team_ama_cum_fireside_chat_1/
----------------------------------------
Submission Title: PlayerUnknown's Battleground
Submission URL: https://www.reddit.com/r/SgGamers/comments/67g8k7/playerunknowns_battleground/
----------------------------------------
Submission Title: If bridge, chess & arnis can be in key Games, then why not E-gaming too?
Submission URL: http://www.straitstimes.com/sport/if-bridge-chess-arnis-can-be-in-key-games-then-why-not-e-gaming-too
----------------------------------------
Submission Title: Xian Kun HO speaks to Straits Times on his interest in representing Singapore at the Asian Games.
Submission URL: http://www.straitstimes.com/sport/local-

Topic 0: pc + going + consider + since + personally
Topic 1: let + https + also + lf + local
Topic 2: still + since + game + brand + use
Topic 3: good + never + things + heard + deleted
Topic 4: https + sure + siblings + hi + know
Topic 5: hard + get + game + even + thing
Topic 6: game + singaporean + players + pro + hard
Topic 7: would + like + career + singaporean + helping
Topic 8: really + games + play + game + rentals
Topic 9: regular + plays + setting + format + looking
Topic 10: need + going + group + together + matchmaking
Topic 11: games + recognized + game + gaming + looking
Topic 12: pretty + building + buy + sure + give
Topic 13: right + shipping + sgd + including + comes
Topic 14: testing + mind + would + shops + hi
Topic 15: objective + hots + different + boils + game
Topic 16: 3 + 1999 + wayback + machine + hopped
Topic 17: dota + hots + think + may + someone
Topic 18: play + one + want + photos + us
Topic 19: arcades + play + pretty + playing + people


In [14]:
# Set the subreddit and keyword
subreddit_names = ['sggaming']
keywords = ['games', 'game', 'GAME', 'videogames', 'racinggame', 'video games', 'raced', 'race', 'league', 'tournament']


comments_and_submissions_results = fetch_comments_and_submissions(subreddit_names, keywords, limit_per_subreddit=None)

# Create a DataFrame from comments_and_submissions_results
comments_df = pd.DataFrame.from_records(comments_and_submissions_results)

# Print the DataFrame
print("Comments and Submissions DataFrame:")
print(comments_df)

# Example usage:
# Assuming you have a DataFrame named comments_df with a 'comments' column
processed_comments_df, common_words_comments = process_comments(comments_df)
print("Processed Comments DataFrame:")
print(processed_comments_df.head())

print("\nMost Common Words in Comments:")
print(common_words_comments.head(50))


# Example usage:
# Assuming you have a DataFrame named commentss_df with a 'comments' column
topics = generate_topics(comments_df, num_topics=2)
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')

# LDA model:
headline_topics = train_lda_model(comments_df['comments'])
for topic in headline_topics:
    print(topic)






Fetching comments and submissions from r/sggaming...
Submission Title: Seeking for participants to complete surveys for Thesis Research on Gaming [Please help🙏]
Submission URL: https://www.reddit.com/r/sggaming/comments/161o2vv/seeking_for_participants_to_complete_surveys_for/
----------------------------------------
Submission Title: Seeking for participants to complete surveys for Thesis Research on Gaming [Please help🙏]
Submission URL: https://www.reddit.com/r/sggaming/comments/13vrkck/seeking_for_participants_to_complete_surveys_for/
----------------------------------------
Submission Title: Find Friends for Games
Submission URL: https://www.reddit.com/r/sggaming/comments/10174qd/find_friends_for_games/
----------------------------------------
Submission Title: Where can i connect to the game industry in Singapore?
Submission URL: https://www.reddit.com/r/sggaming/comments/urh71c/where_can_i_connect_to_the_game_industry_in/
----------------------------------------
Submission Title

In [21]:
# Set the subreddit and keyword
subreddit_names = ['cars']
keywords = ['Singapore', 'singapore']

comments_and_submissions_results = fetch_comments_and_submissions(subreddit_names, keywords, limit_per_subreddit=None)

# Create a DataFrame from comments_and_submissions_results
comments_df = pd.DataFrame.from_records(comments_and_submissions_results)

# Print the DataFrame
print("Comments and Submissions DataFrame:")
print(comments_df)

# Example usage:
# Assuming you have a DataFrame named comments_df with a 'comments' column
processed_comments_df, common_words_comments = process_comments(comments_df)
print("Processed Comments DataFrame:")
print(processed_comments_df.head())

print("\nMost Common Words in Comments:")
print(common_words_comments.head(50))


# Example usage:
# Assuming you have a DataFrame named commentss_df with a 'comments' column
topics = generate_topics(comments_df, num_topics=2)
for topic, top_words in topics.items():
    print(f'{topic}: {top_words}')

# LDA model:
headline_topics = train_lda_model(comments_df['comments'])
for topic in headline_topics:
    print(topic)







Fetching comments and submissions from r/cars...
Submission Title: Crazy Rich Asians & SECRET Car Collections | Capturing Car Culture
Submission URL: https://www.reddit.com/r/cars/comments/1625c4w/crazy_rich_asians_secret_car_collections/
----------------------------------------
Submission Title: 2016 Honda Civic EX: My True Cost Of Ownership
Submission URL: https://www.reddit.com/r/cars/comments/15q0tk2/2016_honda_civic_ex_my_true_cost_of_ownership/
----------------------------------------
Submission Title: The INSANE struggle of owning a car in Singapore | Hagerty
Submission URL: https://www.reddit.com/r/cars/comments/15p9rng/the_insane_struggle_of_owning_a_car_in_singapore/
----------------------------------------
Submission Title: Guess which country had more Mercedes-Benz sold last year than any other brand (and it's not Germany)?
Submission URL: https://www.reddit.com/r/cars/comments/10j9ie4/guess_which_country_had_more_mercedesbenz_sold/
----------------------------------------

Submission Title: Singapore's Lamborghini club shows up for a charity event.
Submission URL: http://youtube.com/watch?v=KuuRlzRJ6Cg
----------------------------------------
Submission Title: Lamborghini Arriva: Inaugural showcase of iconic models in Singapore
Submission URL: http://www.lifestyleasia.com/sg/en/gear/motors/photo-story/lamborghini-arriva-inaugural-showcase-iconic-models/
----------------------------------------
Submission Title: [spotted] The BMW i3 on the road. In Singapore.
Submission URL: https://www.youtube.com/watch?v=b0UNnMKMecA
----------------------------------------
Submission Title: This rare beauty is on display at the Lamborghini Showcase in Singapore
Submission URL: https://www.reddit.com/r/cars/comments/2f6k4z/this_rare_beauty_is_on_display_at_the_lamborghini/
----------------------------------------
Submission Title: If you thought cars in Greece are expensive, wait til you see Singapore's prices....
Submission URL: https://www.reddit.com/r/cars/comments/1y

Topic 1: ['deleted', 'egr', 'recommend', 'runner', 'distinguish']
Topic 2: ['car', 'singapore', 'cars', 'like', 'people']
Topic 0: high + tax + risk + road + import
Topic 1: deleted + cars + would + car + miles
Topic 2: car + like + cars + also + drive
Topic 3: cars + get + car + better + engine
Topic 4: car + get + cars + singapore + us
Topic 5: car + people + get + would + well
Topic 6: orange + removed + people + like + rich
Topic 7: car + coe + singapore + cars + vehicle
Topic 8: get + car + cars + great + even
Topic 9: oil + car + like + used + drive
Topic 10: cars + car + singapore + like + place
Topic 11: car + 1 + cost + people + road
Topic 12: us + california + sure + especially + bad
Topic 13: car + cars + much + singapore + really
Topic 14: cars + people + singapore + also + car
Topic 15: like + one + car + engine + http
Topic 16: like + car + people + would + place
Topic 17: car + singapore + cars + years + year
Topic 18: car + would + snow + singapore + people
Topic 19: ca